# Analyse the bf data

## Aim:

I did a TC experiment and want to get the BF analysis. To this end I run Ilastik + CP segmentation on the timecourse

This is for all the plates from the experiment - thus do an analysis for each of the experiments seperately.

In [ ]:
sm = snakemake

In [ ]:
import os
import pandas as pd
import numpy as np
import re
import plotnine as pn
import string
import pathlib


In [ ]:
import multiprocessing
import functools


In [ ]:
# dat_img columns
COL_SPHERMASK = 'ObjectsFileName_mask'
COL_DATE = 'Metadata_Date'
COL_PLATE = 'Metadata_plate'
COL_REP = 'Metadata_plateloc'
COL_AC = 'Metadata_acquisition'
COL_WELL = 'Metadata_well'
COL_WL = 'Metadata_wl'
COL_DPS = 'Metadata_day'
COL_PLATEAC = 'Metadata_plateac'

COLS_META = [COL_SPHERMASK, COL_DATE, COL_PLATE, COL_REP, COL_WELL, COL_DPS]
# user defined
COL_MAXPROB = 'is_maxprob'
COL_MAXPROBAC = 'is_maxprobac'
COL_HIGHPROB = 'is_highprob'

COL_ISLASTTP = 'is_lasttp'
COL_ISMEDIAN = 'is_mediangood'

COL_SELECTEDSEGMENTAITON = 'is_selected'

COL_CV = 'well_cv'

COL_ROWIDX = 'row_idx'
COL_COLIDX = 'col_idx'
# dat_sphere columns
COL_AREA = 'AreaShape_Area'
COL_IMGNUMBER = 'ImageNumber'
COL_SPHEREPROB = 'Intensity_MeanIntensity_IsSphere'


# dat_layout columns
L_COL_CLIDX = 'cellline_idx'
L_COL_TPIDX = 'timepoint_idx'
L_COL_CTIDX = 'count_idx'
L_COL_PTIDX = 'plate_idx'
L_COL_WELLIDX = 'well_idx'
L_COL_ROWIDX = 'row_idx'
L_COL_COLIDX = 'col_idx'

L_COL_PTID = 'plate_id'
L_COL_CLID = 'cellline_id'
L_COL_CTID = 'count_id'
L_COL_TPID = 'timepoint_id'

L_COL_RLAB = 'row_label'
L_COL_CLAB = 'col_label'
L_COL_WELL = 'well'


fillvals = {COL_REP: 'norm', COL_WL: 1}
colmap_layout_img = {L_COL_PTID: COL_PLATE, L_COL_WELL: COL_WELL}

In [ ]:
%matplotlib inline

In [ ]:
#fn_layout ='../../../experiments/20180530_phys_4celllines-time-size_v1/figures/tc_layout.csv'
fn_hq_spheres = sm.output.fn_hqspheres

In [ ]:
fol_overviews_plates = pathlib.Path(sm.output.fol_plateov)
fol_overviews_wells = pathlib.Path(sm.output.fol_wellov)

In [ ]:
for fol in [fol_overviews_plates, fol_overviews_wells]:
    fol.mkdir(parents=True, exist_ok=True)

In [ ]:
fn_img = sm.input.fn_images
fn_sphere = sm.input.fn_mask

In [ ]:
fol_imgs = sm.input.fol_images
fol_masks = sm.input.fol_masks

In [ ]:
dat_img = pd.read_csv(fn_img)
dat_sphere = pd.read_csv(fn_sphere)
#dat_layout = pd.read_csv(fn_layout)

In [ ]:
dat_sphere.head()

In [ ]:
dat_img = dat_img.fillna(fillvals)

In [ ]:
sm.input.fol_images

In [ ]:
def fix_path(dat, col,fol):
    fol = pathlib.Path(fol)
    dat[col] = dat[col].map(lambda x: fol / pathlib.Path(x).name)

In [ ]:
fix_path(dat_img, 'URL_bfimage', fol_imgs)
fix_path(dat_img, 'ObjectsURL_mask', fol_masks)

# Filenames to be fixed:

In [ ]:
#'date', 'plate', 'rep', 'dps',
#       'ac', 'well', 'wl', 'plateac'


# This is a list of tuples, that describes fixes to the metadata due to naming error:
# ('metaid', value, [('mata1', 'new_value'), ('meta2', 'new_value2'),...])
#p161 (13199), p163 (13200), p165 (13190, 13194), p167 (13192, 13196), p169 (13193, 13197), p171 (13191, 13195)
meta_fix = [
        (COL_PLATEAC,13199, [(COL_DPS, 3)]),
        (COL_PLATEAC,13200, [(COL_DPS, 3)]),
        (COL_PLATEAC,13190, [(COL_DPS, 3)]),
        (COL_PLATEAC,13194, [(COL_DPS, 3)]),
        (COL_PLATEAC,13192, [(COL_DPS, 3)]),
        (COL_PLATEAC,13196, [(COL_DPS, 3)]),
    (COL_PLATEAC,13193, [(COL_DPS, 3)]),
    (COL_PLATEAC,13197, [(COL_DPS, 3)]),
    (COL_PLATEAC,13191, [(COL_DPS, 3)]),
    (COL_PLATEAC,13195, [(COL_DPS, 3)]),
          ]

# all images that have this metadata will be deleted:
meta_drop = [
    ('Metadata_plateac',13186),#Looks like a misslabeled plate
     #        ('Metadata_plateac',12154),#seems wrongly named
      #       ('Metadata_plateac',12157) #wrongly named
            ]

In [ ]:
[c for c in dat_img.columns if c.startswith('URL')]

In [ ]:
def fix_metadata(dat, meta_fix, inplace=False):
    '''
    Corrects (meta)data in a pandas dataframe
    Input:
        dat: a pandas dataframe
        meta fix:  a list of tuples: (key/key, value/values, [('mata1', 'new_value'), ('meta2', 'new_value2'),...])
    Returns:
        the metadata frame with the fixed values
    '''
    if inplace == False:
        dat = dat.copy()
    for meta, val, fixes in meta_fix:
        fil = dat[meta] == val
        col, newval = zip(*fixes)
        dat.loc[fil, col] = newval
    return dat

def drop_data(dat, meta_drop, inplace=False):
    if inplace == False:
        dat = dat.copy()
    for key, val in meta_drop:
        fil = dat[key] != val
        dat = dat.loc[fil, :]
    return dat



In [ ]:
dat_img = fix_metadata(dat_img, meta_fix)
dat_img = drop_data(dat_img, meta_drop)

In [ ]:
def area2diam(x):
    return np.sqrt(x/np.pi)* 0.325*5*2


# Check if the dataset is consistent: allways the same number of images per well? All the correct timepoints?

In [ ]:
d = (dat_img
 #.merge(dat_layout, on=[COL_PLATE, COL_WELL])
 .groupby([COL_PLATE, COL_DATE, COL_DPS, COL_REP,COL_PLATEAC, COL_AC])[COL_IMGNUMBER]
 .describe()

)

In [ ]:
#qgrid.show_grid(d,)
d

# Comments:
 -p163: only imaged 1x @harvesting day 20180205 -> likely a mistake...
 -> rest imaged once forward & once reversed

 - visually confirmed that p161, 20180205 is indeed reversed -> e.g. well B05 is matching
     -> Checked microscope and there is also a normal p161.

 Things to fix:
 - p161 (13199), p163 (13200), p165 (13190, 13194), p167 (13192, 13196), p169 (13193, 13197), p171 (13191, 13195): 20180205: Metadata day should be 3


 # Fix reverted plates

 Some plates have been acquired twice in order to circumvent imaging artefacts (Metadta_plateloc)

 -> For all plates with Plateloc == rev, rotate the wells

 Well
 2 -> 10=12-2
 B -> G = H-B

In [ ]:
old_wells = [ '{}{:02d}'.format(s,n) for s in string.ascii_uppercase[1:7] for n in range(2,12)]
new_wells = [ '{}{:02d}'.format(s,n) for s in reversed(string.ascii_uppercase[1:7]) for n in reversed(range(2,12))]
rev_welldict = {o: n for o,n in zip(old_wells, new_wells)}

In [ ]:
fil = dat_img[COL_REP] == 'rev'
dat_img.loc[fil, COL_WELL] = dat_img.loc[fil, COL_WELL].map(lambda x: rev_welldict[x])

# More sanity checks

 Check uncertainty vs size

In [ ]:
(dat_img
 .merge(dat_sphere, on=COL_IMGNUMBER)
 .query(f'({COL_DPS} != "0")') >>
     pn.ggplot(pn.aes(x='Intensity_MeanIntensity_IsSphere', y=f'area2diam({COL_AREA})'))+
     pn.facet_grid(f'{COL_PLATE}~{COL_DPS}')+
     pn.geom_point()+
 pn.theme(figure_size=(9,12))
)




-> clearly some outlier with low probability & big area!-> Retrain!

For each well, identify the best acquisition by taking the one with the highest average probability

In [ ]:
max_imgs = (dat_img.merge(dat_sphere, on=COL_IMGNUMBER)
     .groupby([COL_WELL, COL_PLATEAC])
     .apply(lambda x: x[COL_IMGNUMBER][x[COL_SPHEREPROB].idxmax()])
)

In [ ]:
dat_img[COL_MAXPROB] = dat_img[COL_IMGNUMBER].isin(max_imgs)

In [ ]:
(dat_img
 .merge(dat_sphere, on=COL_IMGNUMBER)
 .query(f'({COL_DPS} != "0")') >>
     pn.ggplot(pn.aes(x='Intensity_MeanIntensity_IsSphere', y=f'area2diam({COL_AREA})'))+
     pn.facet_grid(f'{COL_PLATE}~{COL_DPS}')+
     pn.geom_point(pn.aes(color=COL_MAXPROB))+
 pn.theme(figure_size=(9,12))
)


# For all wells with multiple acquisition replicates check how they look like

In [ ]:
max_imgs = (dat_img
      .query(f'{COL_MAXPROB} == True')
     .merge(dat_sphere, on=COL_IMGNUMBER)
     .groupby([COL_PLATE,COL_WELL, COL_DPS, COL_AC])
     .apply(lambda x: x[COL_IMGNUMBER][x[COL_SPHEREPROB].idxmax()])
)

dat_img[COL_MAXPROBAC] = dat_img[COL_IMGNUMBER].isin(max_imgs)

In [ ]:
(dat_img
 .merge(dat_sphere, on=COL_IMGNUMBER)
 .query(f'({COL_DPS} != "0")') >>
     pn.ggplot(pn.aes(x='Intensity_MeanIntensity_IsSphere', y=f'area2diam({COL_AREA})'))+
     pn.facet_grid(f'{COL_PLATE}~{COL_DPS}')+
     pn.geom_point(pn.aes(color=COL_MAXPROBAC))+
 pn.theme(figure_size=(9,12))
)

In [ ]:
max_imgs = (dat_img
      .query(f'{COL_MAXPROB} == True')
     .merge(dat_sphere, on=COL_IMGNUMBER)
     .groupby([COL_PLATE,COL_WELL, COL_DPS, COL_AC])
     .apply(lambda x: x[COL_IMGNUMBER][x[COL_SPHEREPROB].idxmax()])
)

dat_img[COL_MAXPROBAC] = dat_img[COL_IMGNUMBER].isin(max_imgs)

In [ ]:
import skimage as sk
import skimage.io
import matplotlib.pyplot as plt
import seaborn as sns
def get_img(imid, dat, col_path):
    path = dat.query(f'{COL_IMGNUMBER} == {imid}')[col_path].values[0]
    img = sk.io.imread(path)
    return img

def get_mask(imid):
    COL_PATHMASK = 'ObjectsURL_mask'
    return get_img(imid, dat_img, COL_PATHMASK)

def get_bfimg(imid):
    COL_PATHMASK = 'URL_bfimage'
    return get_img(imid, dat_img, COL_PATHMASK)

def plot_segmentation(i, color='r', ax=None):
    if ax is None:
        ax = plt.gca()
    ax.imshow(get_bfimg(i))
    ax.contour(get_mask(i), [0.5], linewidths=1.2, colors=color)
    return ax

def plot_segmentation_sb(x, color, **kwargs):
    plot_segmentation(x.values[0])

def plot_segmentation_sb_col(x, col, **kwargs):
    colstr = 'rwb'[int(col.values[0])]
    plot_segmentation(x.values[0], color=colstr)

def plt_well(plateac, well, axs=None):
    imgnrs = dat_img.query(f'{COL_PLATEAC} =={plateac} & {COL_WELL} == "{well}"')[COL_IMGNUMBER]
    ncol = len(imgnrs)
    if axs is None:
        f, axs = plt.subplots(ncols=ncol)
    for a,i in zip(axs,imgnrs):
        plot_segmentation(i, ax=a)
    return axs



In [ ]:
d = ( dat_img
 .query(f'{COL_PLATE} == 176 & {COL_WELL} == "B02" & {COL_MAXPROB} == True')
)

g=sns.FacetGrid(d, row=COL_PLATEAC, col=COL_DPS)
g.map(plot_segmentation_sb,  COL_IMGNUMBER)

In [ ]:
d = ( dat_img
 .query(f'{COL_PLATE} == 165 & {COL_WELL} == "B02" & {COL_MAXPROB} == True')
)

g=sns.FacetGrid(d, row=COL_PLATEAC, col=COL_DPS)
g.map(plot_segmentation_sb,  COL_IMGNUMBER)




-> Attention: the max probability criterion can also be quite good in misssegmented spheres!

- Looks like taking the average in case there were multiple acquisitions (maybe after filtering for a minimum average probability of 0.8) could be approrpiate.

In [ ]:
dat_img = dat_img.drop(COL_HIGHPROB, axis=1,errors='ignore')

t = ((dat_img
     .merge(dat_sphere, on=COL_IMGNUMBER)
     .set_index(COL_IMGNUMBER)[COL_SPHEREPROB] > 0.925)
     .rename(COL_HIGHPROB)
     .reset_index(COL_IMGNUMBER, drop=False)
)
dat_img = dat_img.merge(t)

In [ ]:
dat_img[COL_HIGHPROB].sum()


# Also make a filter for 'last timepoint'

In [ ]:
max_imgs = (dat_img
     .merge(dat_sphere, on=COL_IMGNUMBER)
     .groupby([COL_PLATE,COL_WELL])
     .apply(lambda x: x[COL_IMGNUMBER].loc[x[COL_DPS] == x[COL_DPS].max()])
)

dat_img[COL_ISLASTTP] = dat_img[COL_IMGNUMBER].isin(max_imgs)

In [ ]:
d = (dat_img
 #.merge(dat_layout, on=[COL_PLATE, COL_WELL])
     .query(f'{COL_ISLASTTP} == True')
 .groupby([COL_PLATE, COL_WELL])[COL_HIGHPROB]
 .mean()

)

bad_wells = d[d==0].reset_index()

In [ ]:
bad_wells

In [ ]:
def fix_rowlabels(g):
    for i, axes_row in enumerate(g.axes):
        for j, axes_col in enumerate(axes_row):
            row, col = axes_col.get_title().split('|')

            if i == 0:
                axes_col.set_title(col.strip())
            else:
                axes_col.set_title('')

            if j == 0:
                ylabel = axes_col.get_ylabel()
                axes_col.set_ylabel(row.strip() + ' | ' + ylabel)

def plot_well_imgs(dat_img, plate, well, colorvar=COL_HIGHPROB):
    d = ( dat_img
     .query(f'{COL_PLATE} == {plate} & {COL_WELL} == "{well}" & {COL_MAXPROB} == True')
    )

    g=sns.FacetGrid(d, col=COL_PLATEAC, row=COL_DPS)
    g.map(plot_segmentation_sb_col,  COL_IMGNUMBER, colorvar)
    fix_rowlabels(g)
    g.fig.suptitle(f'{plate} - {well}')
    g.fig.subplots_adjust(top=.9)
    return g

In [ ]:
for p, w in bad_wells[[COL_PLATE, COL_WELL]].values:
    plot_well_imgs(dat_img.query(f'{COL_ISLASTTP} == True'), p, w)
    plt.show()
    plt.close()

In [ ]:
plt.close('all')

-> add to training:
There are some spheres I would add

In [ ]:
selected_from_allbad =     [f'({COL_PLATEAC} == 13230) & ({COL_WELL} == "F05") & ({COL_MAXPROB} == True)']



As a final area take for each sphere the 'median' area, high quality (p>0.9) sphere from the maximal probability wavelength

In [ ]:
def get_median_idx(val):
    # Get the index closest to the median
    if len(val) == 0:
        return None
    val = val.values
    idx = np.argsort(val)
    idxmed = np.nonzero(val[idx] >= np.median(val))
    return idx[idxmed[0][0]]

max_imgs = (dat_img
      .query(f'{COL_MAXPROB} == True')
     .query(f'{COL_HIGHPROB} == True')
     .merge(dat_sphere, on=COL_IMGNUMBER)
     .groupby([COL_PLATE, COL_WELL, COL_DPS])
     .apply(lambda x: x[COL_IMGNUMBER].iloc[get_median_idx(x[COL_AREA])])
)

dat_img[COL_ISMEDIAN] = dat_img[COL_IMGNUMBER].isin(max_imgs)

In [ ]:
(dat_img
      .query(f'{COL_MAXPROB} == True')
     .query(f'{COL_HIGHPROB} == True')
  .query(f'{COL_ISMEDIAN} == True')
     .query(f'({COL_PLATE} == 161) & ({COL_WELL} == "B04")')
     .merge(dat_sphere, on=COL_IMGNUMBER))

In [ ]:
plate = 167
(dat_img
      .query(f'{COL_MAXPROB} == True')
     .query(f'{COL_HIGHPROB} == True')
      .query(f'{COL_ISLASTTP} == True')
     .query(f'({COL_PLATE} == {plate})')
     .merge(dat_sphere, on=COL_IMGNUMBER) >>
pn.ggplot(pn.aes(x=COL_WELL, y=COL_AREA))+
 pn.geom_boxplot()+
 pn.geom_point(pn.aes(color=COL_ISMEDIAN))+
 pn.expand_limits(y=0)+
  pn.theme(axis_text_x = pn.element_text(angle = 90, hjust = 1),
            figure_size=(8,5))+
 pn.ggtitle(str(plate))

)

-> Calculate CV's

-> Still some spheres almost touching that give problems...

-> Calculate CVs to identify problems

In [ ]:
def cv(x):
    x = x.values
    return np.nanstd(x)/np.mean(x)
dat_cv =(dat_img
      .query(f'{COL_MAXPROB} == True')
     .query(f'{COL_HIGHPROB} == True')
       .query(f'{COL_ISLASTTP} == True')
     .merge(dat_sphere, on=COL_IMGNUMBER)
    .groupby([COL_PLATE, COL_DPS, COL_WELL])[COL_AREA]
     .apply(cv)
     .rename(COL_CV)
    .reset_index())

(dat_cv >>
 pn.ggplot(pn.aes(x=COL_CV))+
 pn.facet_grid(f'{COL_PLATE}~{COL_DPS}')+
 pn.geom_histogram()

)

->  All looks good!

Still look at all the spheres with a CV of > 15%

In [ ]:
pdat = dat_cv.query(f'{COL_CV} > 0.1')
pdat

In [ ]:
for p, well in pdat[[COL_PLATE, COL_WELL]].values:
    plot_well_imgs(dat_img.query(f'{COL_ISLASTTP} == True'), p, well, colorvar=COL_ISMEDIAN)
    plt.show()
    plt.close()

Above shows in white the segmentation that is picked for quantification - it actually looks really good.  
All the selected spheres are reasonably segmented.


In [ ]:
unselect_well = []
select_well = [
              ]

In [ ]:
dat_img[COL_SELECTEDSEGMENTAITON] = dat_img[COL_ISMEDIAN].copy()

In [ ]:
for u, s in zip(unselect_well, select_well):
    dat_img.loc[dat_img.query(u).index, COL_SELECTEDSEGMENTAITON] = False
    dat_img.loc[dat_img.query(s).index, COL_SELECTEDSEGMENTAITON] = True

for s in selected_from_allbad:
    # selected from the wells which had no 'good' segmentation
    dat_img.loc[dat_img.query(s).index, COL_SELECTEDSEGMENTAITON] = True

 Finally: make whole plate overviews, allways showing the selected segmentation

 In cases where there was no selected segmentation plot the COL_MAXPROBAX

In [ ]:
def well2colidx(w):
    return int(w[1:])
def well2rowidx(w):
    return string.ascii_uppercase.index(w[0])
dat_img[COL_COLIDX] = dat_img[COL_WELL].map(well2colidx)
dat_img[COL_ROWIDX] = dat_img[COL_WELL].map(well2rowidx)

In [ ]:
def get_best(x):
    if np.any(x[COL_SELECTEDSEGMENTAITON]):
        return x.loc[x[COL_SELECTEDSEGMENTAITON].idxmax]
    else:
        return x.loc[x[COL_MAXPROBAC].idxmax]

In [ ]:
dat_bestimgs = (dat_img
     .query(f'{COL_ISLASTTP} == True')
     .groupby([COL_PLATE, COL_WELL])
     .apply(get_best)
)
dat_bestimgs[COL_PLATE].unique()

In [ ]:
def plot_plate(dat, plate, colorvar):
    pdat = dat.query(f'{COL_PLATE} == {plate}')
    if pdat[COL_WELL].duplicated().any():
        raise ValueError('Duplicated Wells not supported')
    g=sns.FacetGrid(pdat, col=COL_COLIDX, row=COL_ROWIDX)
    g.map(plot_segmentation_sb_col,  COL_IMGNUMBER, colorvar)
    g.fig.suptitle(f'{plate}', size=30)
    g.fig.subplots_adjust(top=.9)
    return g

In [ ]:
def plot_plate_best(plate):
    p = plot_plate(dat_bestimgs, plate, COL_SELECTEDSEGMENTAITON)
    p.savefig(fol_overviews_plates /f'segmented_overview_p{plate}.png')
    #plt.show()
    plt.close()
    
with multiprocessing.Pool(sm.threads) as pool:
    pool.map(plot_plate_best, dat_bestimgs[COL_PLATE].unique())

In [ ]:
dat_hqspheres = (dat_img
      .query(f'{COL_ISLASTTP} == True')
     .query(f'{COL_SELECTEDSEGMENTAITON} == True')
     .merge(dat_sphere, on=COL_IMGNUMBER))

In [ ]:
dat_hqspheres.to_csv(fn_hq_spheres)

Plot all spheres from plate for manual qc

In [ ]:
def plot_well_overviews(folder, plateid):
    folder = pathlib.Path(folder)
    wells = dat_img[COL_WELL].unique()
    for w in wells:
        g = plot_well_imgs(dat_img.query(f'{COL_ISLASTTP} == True'), plateid, w, colorvar=COL_ISMEDIAN)
        g.savefig(folder / f'plate{plateid}_well{w}.png')
        plt.close()

In [ ]:
p='173'
well='B02'
g = plot_well_imgs(dat_img.query(f'{COL_ISLASTTP} == True'), p, well, colorvar=COL_ISMEDIAN)

In [ ]:
import itertools

In [ ]:
def plot_and_save_well(plateid, well, imgdat, folder):
    g = plot_well_imgs(imgdat, plateid, well, colorvar=COL_ISMEDIAN)
    g.savefig(folder / f'plate{plateid}_well{well}.png')
    plt.close()
        
plates=['169', '163', '171', '167', '161', '165', '173', '176']
wells=dat_img[COL_WELL].unique()

with multiprocessing.Pool(sm.threads) as pool:
    pool.starmap(functools.partial(plot_and_save_well,
                                   folder=fol_overviews_wells,
                                   imgdat=dat_img.query(f'{COL_ISLASTTP} == True')),
                 itertools.product(plates, wells))


# Load QC results form 2_manual_qc

-> Do qc and run 2_manual_qc

In [ ]:
fn_qc_res = pathlib.Path(sm.input.fn_manual_qc)

In [ ]:
sel_plate = 176

In [ ]:
dat_qcres = pd.read_csv(fn_qc_res)

In [ ]:
dat_img.columns

In [ ]:
dat_qcres.columns

In [ ]:
pdat = dat_bestimgs.reset_index(drop=True).merge(dat_qcres, left_on=[COL_WELL, COL_PLATE], right_on=['well', 'plate'])

In [ ]:
COL_ISHQ = 'ishq'

In [ ]:
pdat[COL_ISHQ] = pdat[COL_SELECTEDSEGMENTAITON] & (pdat['bf_qc'] == 'good')

In [ ]:
fig = plot_plate(pdat, sel_plate, COL_ISHQ)

In [ ]:
import skimage.filters as skif

In [ ]:
ref_img = fig.axes.flatten()[1].images[0].get_array().data

In [ ]:
for ax in fig.axes.flatten():
    ax.axis('off')
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    aximg = ax.images[0]
    img = aximg.get_array().data.copy()
    p = 99.95
    maxperc = np.percentile(img, p)
    minperc = np.percentile(img, 100-p)
    img[ img > maxperc]=maxperc
    img[ img < minperc]=minperc
    #img = img-minperc
    aximg.set_cmap('gist_gray')
    #img = sk.transform.match_histograms(aximg.get_array().data, ref_img)
    #aximg.norm(img)
    aximg.set_data(img)
    aximg.set_clim((minperc,maxperc))

In [ ]:
fig.fig.suptitle('')

In [ ]:
fig.fig

In [ ]:
fig.savefig(sm.output.fn_fig1_bfspheres)